In [21]:
from janome.tokenizer import Tokenizer

# 開始、終端記号を設定する。
BEGIN = '__BEGIN__'
END = '__END__'
# 実際に一文を3単語ずつに分けてみる。
sentence = 'おいしいビールを飲もう'

t = Tokenizer()
# ここで型変換しないとgeneratorのリストができてしまう
# 分かち書きした文章に開始終端記号を追加してリストにする
words = list(t.tokenize(sentence, wakati=True))
words = [BEGIN] + words + [END]

three_words_list = []
for i in range(len(words) - 2):
    three_words_list.append(words[i:i + 3])
three_words_list

[['__BEGIN__', 'おいしい', 'ビール'],
 ['おいしい', 'ビール', 'を'],
 ['ビール', 'を', '飲も'],
 ['を', '飲も', 'う'],
 ['飲も', 'う', '__END__']]

In [20]:
from collections import Counter

def get_three_words_list(sentence):
    """文章を3単語の組にして返す"""
    t = Tokenizer()
    words = list(t.tokenize(sentence, wakati=True))
    words = [BEGIN] + words + [END]
    three_words_list = []
    for i in range(len(words) - 2):
        three_words_list.append(tuple(words[i:i+3]))
    return three_words_list

sentences = ['おいしいビールを飲もう', 'ビールを飲もう', 'おいしいビールは生']
three_words_list = []
for sentence in sentences:
    three_words_list += get_three_words_list(sentence)
three_words_count = Counter(three_words_list)
three_words_count


Counter({('__BEGIN__', 'おいしい', 'ビール'): 2,
         ('おいしい', 'ビール', 'を'): 1,
         ('ビール', 'を', '飲も'): 2,
         ('を', '飲も', 'う'): 2,
         ('飲も', 'う', '__END__'): 2,
         ('__BEGIN__', 'ビール', 'を'): 1,
         ('おいしい', 'ビール', 'は'): 1,
         ('ビール', 'は', '生'): 1,
         ('は', '生', '__END__'): 1})

In [ ]:
def generate_marcov_dict(three_words_count):
    """マルコフ連鎖での文章作成用の辞書データを作成する"""
    markov_dict = {}
    for three_words, count in three_words_count.items():
        two_words = three_words[:2]
        next_words = three_words[2]
        if two_words not in markov_dict:
            markov_dict[two_words] = {'words': [], 'weights': []}
        markov_dict[two_words]